In [29]:
# import relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import os
import glob
import re
import requests
from datetime import datetime
from urllib.parse import urlparse, parse_qs, unquote
from collections import deque
from bs4 import BeautifulSoup
from tqdm import tqdm
import ast

In [30]:
# if code was already run, to save time
logs_df = pd.read_csv("data/clean_logs.csv")
ark_df = pd.read_csv("data/ark_df.csv")
non_empty_arks_df = pd.read_csv("arks_final/arks_non_empty.csv")
empty_arks_df = pd.read_csv("arks_final/arks_empty.csv")

## 1 : Extracting the logs

In [2]:
# get logs for month of February
folder_path = r'../../../lhstdata1/students/Gallica_logs/1LogGallicaFevrier2016'
all_files = glob.glob(folder_path + "/*.log.gz")
all_files.sort(key=lambda f: int(re.sub('\D', '', f)))

In [3]:
# take only part of the data, to run tests on a smaller batch for speed
all_files = all_files[0:80]
# empty dataframe to store concatenation
dfs = []
for f in all_files:
    try:
        # read csv file into df
        df = pd.read_csv(f, encoding='UTF-8', sep ='\t', engine='python', header=None)
        # append to list of dfs
        dfs.append(df)
    except Exception as e:
        print(f"Error loading file {f}: {e}")

#from the list, concatenate the dataframes into a single one
combined_df = pd.concat(dfs, ignore_index=True)

In [4]:
combined_df.shape

(10042040, 1)

## 2 : Extracting features from logs 

In [5]:
# split the lines to recover meaningful information
lines_df = combined_df[0].str.split('##', expand=True)

In [6]:
lines_df.head()

,0,1,2,3,4
0,,e7fdec50f50253f6796d61b5382155f8,null,null,"- - [31/Jan/2016:18:59:19 +0100] ""GET /ark:/12..."
1,,5d1edaaaa9cf4772ce828127ce3e523a,United States,Chapel Hill,"- - [31/Jan/2016:18:59:20 +0100] ""GET /ark:/12..."
2,,fd91d2df98948719be32c40d684662fe,Mexico,Mexico City,"- - [31/Jan/2016:18:59:21 +0100] ""GET /service..."
3,,fd91d2df98948719be32c40d684662fe,Mexico,Mexico City,"- - [31/Jan/2016:18:59:21 +0100] ""GET /service..."
4,,fd91d2df98948719be32c40d684662fe,Mexico,Mexico City,"- - [31/Jan/2016:18:59:21 +0100] ""GET /service..."


In [7]:
# extracting dates
temp = pd.DataFrame()
temp['Date'] = lines_df.apply(lambda x: x[4].split("]")[0].split("[")[1], axis=1)
# convert to datetime format for sorting
# temp['Date'] = pd.to_datetime(temp['Date'], format='%d/%b/%Y:%H:%M:%S %z')
# sort by ascending order
# temp = temp.sort_values(by='Date', ascending=True)

In [8]:
temp

,Date
0,31/Jan/2016:18:59:19 +0100
1,31/Jan/2016:18:59:20 +0100
2,31/Jan/2016:18:59:21 +0100
3,31/Jan/2016:18:59:21 +0100
4,31/Jan/2016:18:59:21 +0100
...,...
10042035,02/Feb/2016:05:30:18 +0100
10042036,02/Feb/2016:05:30:19 +0100
10042037,02/Feb/2016:05:30:24 +0100
10042038,02/Feb/2016:05:30:24 +0100


In [9]:
# we have logs from 31/01/2016 - 13h to 02/02/2016 - 5h30, a period of over 24h

In [10]:
# extracting request
temp['Request'] = lines_df.apply(lambda x: ' '.join(x[4].split("\"")[1].split(' ')[:2]), axis=1)

In [11]:
# extracting referrer
temp['Referrer'] = lines_df.apply(lambda x: x[4].split("\"")[3], axis=1)

In [12]:
#extract request
temp['Request'] = lines_df.apply(lambda x: ' '.join(x[4].split("\"")[1].split(' ')[:2]), axis=1)

In [13]:
# function to extract ark from the request
def extract_ark(request):
    # init ark variable
    ark = '-'
    # regular expression pattern, 12148 is specific to gallica
    pattern = r'/12148/([^/.]+)'
    # use regec to find ark in request string
    match = re.search(pattern, request)
    if match:
        ark = match.group(1)
    return ark

temp['Ark'] = temp.apply(lambda x: extract_ark(x['Request']), axis=1)

In [14]:
# rename the columns with informative names
lines_df = lines_df.rename(columns = {1:"IPaddress", 2:"Country", 3:"City", 4:"Full_request"})

In [15]:
logs_df = pd.concat([lines_df, temp], axis=1)

In [16]:
logs_df.head(10)

,0,IPaddress,Country,City,Full_request,Date,Request,Referrer,Ark
0,,e7fdec50f50253f6796d61b5382155f8,null,null,"- - [31/Jan/2016:18:59:19 +0100] ""GET /ark:/12...",31/Jan/2016:18:59:19 +0100,GET /ark:/12148/bpt6k70211m,-,bpt6k70211m
1,,5d1edaaaa9cf4772ce828127ce3e523a,United States,Chapel Hill,"- - [31/Jan/2016:18:59:20 +0100] ""GET /ark:/12...",31/Jan/2016:18:59:20 +0100,GET /ark:/12148/bpt6k622723.planchecontact.r=e...,-,bpt6k622723
2,,fd91d2df98948719be32c40d684662fe,Mexico,Mexico City,"- - [31/Jan/2016:18:59:21 +0100] ""GET /service...",31/Jan/2016:18:59:21 +0100,GET /services/ajax/extract/ark:/12148/bpt6k866...,http://gallica.bnf.fr/services/engine/search/s...,bpt6k866717
3,,fd91d2df98948719be32c40d684662fe,Mexico,Mexico City,"- - [31/Jan/2016:18:59:21 +0100] ""GET /service...",31/Jan/2016:18:59:21 +0100,GET /services/engine/search/sru?operation=sear...,http://gallica.bnf.fr/services/engine/search/s...,-
4,,fd91d2df98948719be32c40d684662fe,Mexico,Mexico City,"- - [31/Jan/2016:18:59:21 +0100] ""GET /service...",31/Jan/2016:18:59:21 +0100,GET /services/ajax/extract/ark:/12148/bpt6k653...,http://gallica.bnf.fr/services/engine/search/s...,bpt6k65309826
5,,fd91d2df98948719be32c40d684662fe,Mexico,Mexico City,"- - [31/Jan/2016:18:59:21 +0100] ""GET /service...",31/Jan/2016:18:59:21 +0100,GET /services/ajax/extract/ark:/12148/bpt6k653...,http://gallica.bnf.fr/services/engine/search/s...,bpt6k6530977w
6,,83bbb4ec83c93384666a2884238bbd55,United States,Menlo Park,"- - [31/Jan/2016:18:59:22 +0100] ""GET /assets/...",31/Jan/2016:18:59:22 +0100,GET /assets/static/javascripts/application/vie...,-,-
7,,83bbb4ec83c93384666a2884238bbd55,United States,Menlo Park,"- - [31/Jan/2016:18:59:22 +0100] ""GET /assets/...",31/Jan/2016:18:59:22 +0100,GET /assets/static/javascripts/application/bui...,-,-
8,,83bbb4ec83c93384666a2884238bbd55,United States,Menlo Park,"- - [31/Jan/2016:18:59:22 +0100] ""GET /assets/...",31/Jan/2016:18:59:22 +0100,GET /assets/static/javascripts/application/ent...,-,-
9,,83bbb4ec83c93384666a2884238bbd55,United States,Menlo Park,"- - [31/Jan/2016:18:59:22 +0100] ""GET /assets/...",31/Jan/2016:18:59:22 +0100,GET /assets/static/javascripts/application/ent...,-,-


In [17]:
# function to extract search terms from the request

def extract_search_terms(request):
    # Parse the URL
    parsed_url = urlparse(request)
    
    # Extract the query parameters
    query_params = parse_qs(parsed_url.query)
    
    # Extract the search query from the query parameters
    search_query = query_params.get('query', [''])[0]
    
    # URL-decode the search query
    search_query = unquote(search_query)
    
    # Extract search terms using regular expression
    search_terms = re.findall(r'"([^"]+)"', search_query)
    
    return search_terms

In [18]:
logs_df['search_terms'] = logs_df.apply(lambda x: extract_search_terms(x['Request']), axis=1)

## 3 : Enrich data

We want to enrich the data by adding additional information about the requested document (if there was one) : Dewey class, type of document. This will help create a diversity metric for the sessions later on.

In [20]:
# get list of unique arks
logs_df.loc[logs_df['Ark'] == '-', 'Ark'] = None
ark_df = logs_df[logs_df['Ark'].notnull()].copy()
ark_df.drop_duplicates(subset=['Ark'], inplace=True)

In [24]:
# create visibility metric for each ark
# calculate the frequency of each ark
ark_counts = logs_df['Ark'].value_counts()

# Calculate the visibility for each value in 'ark'
logs_df['visibility'] = logs_df['Ark'].map(lambda x: ark_counts.get(x, 0) / len(logs_df))

In [27]:
logs_df.to_csv("data/clean_logs.csv")

In [30]:
ark_df['Ark'].to_csv("data/ark_df.csv")

In [31]:
# OAI request to Gallica
def OAI(id):

    OAI_BASEURL = 'https://gallica.bnf.fr/services/OAIRecord?ark='

    url = "".join([OAI_BASEURL, id])

    s = requests.get(url, stream=True)
    soup = BeautifulSoup(s.content,"lxml-xml")
    return soup


def extract_metadata(ark):
    if ark == '-':
        return ark, None, None
    theme = ''
    typedoc = ''
    try:
        oai_result = OAI(ark)
        if oai_result is not None:
            # Extracting theme
            sdewey_tag = oai_result.find("sdewey")
            if sdewey_tag:
                theme = sdewey_tag.text
        
            # Extracting typedoc
            typedoc_tag = oai_result.find("typedoc")
            if typedoc_tag:
                typedoc = typedoc_tag.text
        
    except Exception as e:
        print("Error occurred:", e)

    return ark, theme, typedoc

In [32]:
# function to request the arks metadata, with a progress bar and saving batches
def apply_with_progress(df, func, result_df, batch_size=20000, output_prefix='arks_temp/arks'):
    result = []
    with tqdm(total=len(df)) as pbar:
        for index, row in df.iterrows():
            curr_result = func(row)
            result.append(curr_result)
            pbar.update(1)
            
            # Save to CSV file every batch_size ARKs
            if len(result) % batch_size == 0:
                batch_df = pd.DataFrame(result, columns=['Ark', 'Theme', 'Type'])
                batch_df.to_csv(f'{output_prefix}_{len(result)}.csv', index=False)
    
    return pd.DataFrame(result, columns=['Ark', 'Theme', 'Type'])

In [ ]:
# takes VERY long, do not run if arks' types and dewey classes are already computed
result_df_ark = pd.DataFrame()
temp_ark_df = ark_df
result_df = apply_with_progress(temp_ark_df, lambda row: extract_metadata(row['Ark']), result_df_ark)

# Save the remaining results to CSV
result_df.to_csv('arks_final/all_arks.csv', index=False)

## 4: Concatenating arks and correcting them

In [33]:
# combine all arks and check if some are empty or didn't work
arks1_100000_df = pd.read_csv("arks_final/arks1_100000.csv")
arks2_100000_df = pd.read_csv("arks_final/arks2_100000.csv")
arks3_100000_df = pd.read_csv("arks_final/arks3_100000.csv")
arks4_100000_df = pd.read_csv("arks_final/arks4_100000.csv")
arks5_df = pd.read_csv("arks_final/arks5.csv")
arks_run2_df = pd.read_csv("arks_final/empty_arks.csv")
arks_run2_nonnull_df = arks_run2_df[arks_run2_df['Type'].notnull()].copy()

In [34]:
arks_df = pd.concat([arks1_100000_df, arks2_100000_df, arks3_100000_df, arks4_100000_df, arks5_df, arks_run2_nonnull_df], ignore_index=True)
arks_df = arks_df.sort_values('Type', na_position='last').drop_duplicates('Ark', keep='first')
arks_df.reset_index(drop=True, inplace=True)

In [35]:
arks_df

,Ark,Theme,Type
0,btv1b550002388,NaN,carte
1,btv1b8458668k,NaN,carte
2,btv1b8495083c,NaN,carte
3,btv1b530351956,NaN,carte
4,btv1b53016832r,NaN,carte
...,...,...,...
477986,bpt6k815278%20,NaN,NaN
477987,bpt6k6451820c%20,NaN,NaN
477988,bpt6k6464419j%20,NaN,NaN
477989,bpt6k62515285%20,NaN,NaN


In [36]:
# seperate empty and non empty arks
non_empty_arks_df = arks_df[pd.notnull(arks_df['Type'])]
# find empty typedoc and create new dataframe to request these arks
empty_arks_df = arks_df[pd.isnull(arks_df['Type'])]

In [37]:
nb_empty_arks = len(empty_arks_df)
print("There are",nb_empty_arks , "empty arks, which is", nb_empty_arks/len(arks_df)*100, "% of the arks.")

There are 18170 empty arks, which is 0.03801326803224329 % of the arks.


In [38]:
# save empty and non empty arks for now
non_empty_arks_df.to_csv("arks_final/arks_non_empty.csv")
empty_arks_df.to_csv("arks_final/arks_empty.csv")

In [39]:
# check format of these arks, correct it if necessary, re-request them
empty_arks_df

,Ark,Theme,Type
459821,btv1b7720377q,NaN,NaN
459822,bpt6k5795138h,NaN,NaN
459823,bpt6k298770b,NaN,NaN
459824,btv1b5966539r,NaN,NaN
459825,bpt6k5489230s,NaN,NaN
...,...,...,...
477986,bpt6k815278%20,NaN,NaN
477987,bpt6k6451820c%20,NaN,NaN
477988,bpt6k6464419j%20,NaN,NaN
477989,bpt6k62515285%20,NaN,NaN


In [42]:
# seems some of them are not cut at the right place at end, we fix that then re-request
# Function to clean arks
def clean_ark(ark):
    # Remove any query parameters
    ark = re.sub(r'\?.*$', '', ark)
    # Remove any trailing non-alphanumeric characters
    ark = re.sub(r'[^a-zA-Z0-9]$', '', ark)
    ark = re.sub(r'%20$', '', ark)
    return ark


empty_arks_df = empty_arks_df.copy()

# Apply cleaning function to 'Ark' column using .loc to avoid SettingWithCopyWarning
empty_arks_df.loc[:, 'Ark_clean'] = empty_arks_df['Ark'].apply(clean_ark)

In [43]:
empty_arks_df

,Ark,Theme,Type,Ark_clean
459821,btv1b7720377q,NaN,NaN,btv1b7720377q
459822,bpt6k5795138h,NaN,NaN,bpt6k5795138h
459823,bpt6k298770b,NaN,NaN,bpt6k298770b
459824,btv1b5966539r,NaN,NaN,btv1b5966539r
459825,bpt6k5489230s,NaN,NaN,bpt6k5489230s
...,...,...,...,...
477986,bpt6k815278%20,NaN,NaN,bpt6k815278
477987,bpt6k6451820c%20,NaN,NaN,bpt6k6451820c
477988,bpt6k6464419j%20,NaN,NaN,bpt6k6464419j
477989,bpt6k62515285%20,NaN,NaN,bpt6k62515285


In [48]:
# we also know some documents are not available anymore so the request won't work, so we will have some missing ark values
arks_run2_df = pd.DataFrame()
clean_arks_df = apply_with_progress(empty_arks_df[10000:], lambda row: extract_metadata(row['Ark_clean']), arks_run2_df, 10000, 'arks_temp/fill_arks.csv')

# Save the remaining results to CSV
clean_arks_df.to_csv('arks_final/filling_arks.csv', index=False)

 12%|████▊                                    | 949/8170 [17:27<53:28,  2.25it/s]

Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


 12%|████▊                                    | 953/8170 [17:28<25:30,  4.71it/s]

Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


 12%|████▊                                    | 955/8170 [17:28<20:32,  5.85it/s]

...

100%|█████████████████████████████████████▉| 8166/8170 [1:57:19<00:00,  5.96it/s]

Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


100%|█████████████████████████████████████▉| 8168/8170 [1:57:19<00:00,  5.72it/s]

Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error occurred: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


100%|██████████████████████████████████████| 8170/8170 [1:57:20<00:00,  1.16it/s]


In [45]:
# now concatenate the obtained arks with the rest
arks_final_df = pd.concat([non_empty_arks_df, clean_arks_df], ignore_index=True)

In [46]:
arks_final_df

,Ark,Theme,Type
0,btv1b550002388,NaN,carte
1,btv1b8458668k,NaN,carte
2,btv1b8495083c,NaN,carte
3,btv1b530351956,NaN,carte
4,btv1b53016832r,NaN,carte
...,...,...,...
477986,bpt6k815278,,
477987,bpt6k6451820c,,
477988,bpt6k6464419j,,
477989,bpt6k62515285,,


In [47]:
arks_final_df.to_csv("data/arks_final.csv")